In [1]:
import pandas as pd, numpy as np

# Compile regression estiamtes / examine for logical consistency

In [2]:
w_metadata_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/lsff_input_coverage_data_with_metadata.csv'


In [3]:
w_metadata_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/lsff_input_coverage_data_with_metadata.csv'
df = pd.read_csv(w_metadata_path)

In [4]:
df.estimation_status.unique()

array([nan, 'multiplicative', 'na', 'lack of evidence', 'regression'],
      dtype=object)

In [5]:
A = df[(df.value_description=="percent of population eating vehicle")].rename(columns={'value_mean':"A",'estimation_status':'A_estimation_status'})
B = df[(df.value_description=="percent of population eating industrially produced vehicle")].rename(columns={'value_mean':"B",'estimation_status':'B_estimation_status'})
C = df[(df.value_description=="percent of population eating fortified vehicle")].rename(columns={'value_mean':"C",'estimation_status':'C_estimation_status'})

In [6]:
A = A.drop(columns=['value_description','nutrient'])
B = B.drop(columns=['value_description','nutrient'])
C = C.drop(columns=['value_description'])

In [7]:
ref = A.merge(B, on = ['location_id','location_name','vehicle'], how = 'left').merge(C, on = ['location_id','location_name','vehicle'], how = 'outer')[['location_name','vehicle','nutrient','A','A_estimation_status','B','B_estimation_status','C','C_estimation_status']]


In [8]:
ref

,location_name,vehicle,nutrient,A,A_estimation_status,B,B_estimation_status,C,C_estimation_status
0,Bangladesh,oil,vitamin a,100.0,na,88.400000,na,42.849572,multiplicative
1,Bangladesh,wheat flour,folic acid,41.9,na,38.241333,multiplicative,0.155875,multiplicative
2,Bangladesh,wheat flour,iron,41.9,na,38.241333,multiplicative,0.155875,multiplicative
3,Bangladesh,wheat flour,vitamin a,41.9,na,38.241333,multiplicative,0.155875,multiplicative
4,Bangladesh,wheat flour,zinc,41.9,na,38.241333,multiplicative,0.155875,multiplicative
...,...,...,...,...,...,...,...,...,...
137,Nigeria,oil,vitamin a,NaN,NaN,NaN,NaN,7.463576,NaN
138,Nigeria,wheat flour,folic acid,NaN,NaN,NaN,NaN,16.799661,NaN
139,Nigeria,wheat flour,iron,NaN,NaN,NaN,NaN,16.799661,NaN
140,Nigeria,wheat flour,vitamin a,NaN,NaN,NaN,NaN,16.799661,NaN


In [9]:
oil_est_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_oil_regression_estimates_3_23_2021.csv'
wheat_est_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_wheat_regression_estimates_3_22_2021.csv'
maize_est_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_maize_regression_estimates_3_22_2021.csv'

ip_wheat_path =  '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortifiable_wheat_regression_estimates_3_31_2021.csv'
ip_oil_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortifiable_oil_regression_estimates_3_31_2021.csv'
ip_maize_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortifiable_maize_regression_estimates_3_31_2021.csv'

fort_wheat_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortified_wheat_regression_estimates_3_31_2021.csv'
fort_maize_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortified_maize_regression_estimates_3_31_2021.csv'
fort_oil_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortified_oil_regression_estimates_3_31_2021.csv'

In [10]:
paths_A = [oil_est_path,wheat_est_path,maize_est_path]
paths_B = [ip_wheat_path,ip_oil_path,ip_maize_path]
paths_C = [fort_wheat_path, fort_maize_path, fort_oil_path]

def format_data(paths, var, n):
    draws = [f'draw_{i}' for i in range(n)]
    df = pd.concat([pd.read_csv(path) for path in paths])
    df[var] = df[draws].mean(axis=1)
    df = df.drop(columns=draws)
    
    return df.groupby(['location_name','vehicle']).mean().reset_index()

In [11]:
eats = format_data(paths_A, 'reg_A', n = 1_000)
ip = format_data(paths_B, 'reg_B', n = 500)
fort = format_data(paths_C, 'reg_C',n = 500)

In [31]:
fort

,location_name,vehicle,reg_C
0,Angola,maize flour,11.000013
1,Angola,oil,37.195681
2,Angola,wheat flour,26.257859
3,Bangladesh,oil,87.950000
4,Bangladesh,wheat flour,38.241333
...,...,...,...
57,United Republic of Tanzania,maize flour,19.550000
58,United Republic of Tanzania,oil,73.300000
59,United Republic of Tanzania,wheat flour,50.500000
60,Viet Nam,oil,90.000000


In [34]:
test = eats.merge(ip, on = ['location_name','vehicle'], how = 'outer').merge(fort, on = ['location_name','vehicle'], how = 'outer')

In [36]:
test[(test.reg_A < test.reg_B) |(test.reg_B < test.reg_C)]

,location_name,vehicle,reg_A,reg_B,reg_C
3,Bangladesh,oil,79.200000,87.950000,87.950000
11,China,oil,57.651373,63.503924,NaN
13,Côte d'Ivoire,maize flour,84.958352,11.160705,11.160705
20,Egypt,oil,54.973297,70.424358,NaN
23,Ethiopia,oil,29.350000,41.333333,41.333333
26,Ghana,oil,96.275000,82.099486,82.099486
37,Madagascar,wheat flour,45.179608,31.578540,31.578540
43,Nepal,oil,99.900000,88.793799,88.793799
44,Nepal,wheat flour,27.966667,39.142857,39.142857
47,Niger,wheat flour,41.424994,34.862101,34.862101


In [37]:
test.loc[(test.location_name.isin(['Angola','China','Ghana','Niger','Egypt','Sudan','Madagascar']))].sort_values(['location_name','vehicle'])

,location_name,vehicle,reg_A,reg_B,reg_C
0,Angola,maize flour,51.100278,11.000013,11.000013
1,Angola,oil,90.384708,37.195681,37.195681
2,Angola,wheat flour,27.171970,26.257859,26.257859
11,China,oil,57.651373,63.503924,NaN
12,China,wheat flour,81.832190,53.665634,53.665634
19,Egypt,maize flour,74.363121,21.683282,21.683282
20,Egypt,oil,54.973297,70.424358,NaN
21,Egypt,wheat flour,88.484772,68.055038,68.055038
25,Ghana,maize flour,82.230725,19.936771,19.936771
26,Ghana,oil,96.275000,82.099486,82.099486


In [12]:
# impute A
ref = ref.merge(eats, on = ['location_name','vehicle'], how = 'left')
ref.loc[(ref.A.isna()),'A'] = ref.reg_A

In [13]:
ref

,location_name,vehicle,nutrient,A,A_estimation_status,B,B_estimation_status,C,C_estimation_status,reg_A
0,Bangladesh,oil,vitamin a,100.00,na,88.400000,na,42.849572,multiplicative,79.200000
1,Bangladesh,wheat flour,folic acid,41.90,na,38.241333,multiplicative,0.155875,multiplicative,62.233333
2,Bangladesh,wheat flour,iron,41.90,na,38.241333,multiplicative,0.155875,multiplicative,62.233333
3,Bangladesh,wheat flour,vitamin a,41.90,na,38.241333,multiplicative,0.155875,multiplicative,62.233333
4,Bangladesh,wheat flour,zinc,41.90,na,38.241333,multiplicative,0.155875,multiplicative,62.233333
...,...,...,...,...,...,...,...,...,...,...
137,Nigeria,oil,vitamin a,98.50,NaN,NaN,NaN,7.463576,NaN,98.500000
138,Nigeria,wheat flour,folic acid,49.05,NaN,NaN,NaN,16.799661,NaN,49.050000
139,Nigeria,wheat flour,iron,49.05,NaN,NaN,NaN,16.799661,NaN,49.050000
140,Nigeria,wheat flour,vitamin a,49.05,NaN,NaN,NaN,16.799661,NaN,49.050000


In [14]:
df = ref.copy()

In [15]:
# merge on B and filter
est = pd.concat([pd.read_csv(path) for path in paths_B])

est = est.groupby(['location_name','vehicle']).mean().reset_index()

df = df.merge(est, on = ['location_name','vehicle'], how = 'left')

draws = [f'draw_{i}' for i in range(500)]

for draw in draws:
    df.loc[(df.A < df[draw]) | 
                 (df.C > df[draw]),draw] = np.nan

df['reg_B'] = df[draws].mean(axis=1)
df = df.drop(columns = draws)

# these are rows you still dont have estimates for
# df[(df.B.isna()) & (df.reg_B.isna())]

df.loc[(df.B.isna()) | (df.B_estimation_status=='regression'),'B'] = df.reg_B

In [16]:
df[(df.A < df.B) | (df.B < df.C)]

,location_name,vehicle,nutrient,A,A_estimation_status,B,B_estimation_status,C,C_estimation_status,reg_A,reg_B
45,Burkina Faso,oil,vitamin a,92.23,na,51.975594,multiplicative,75.000000,na,92.23,NaN
46,Burkina Faso,wheat flour,folic acid,48.00,na,52.300000,multiplicative,32.164500,multiplicative,52.30,NaN
47,Burkina Faso,wheat flour,iron,48.00,na,52.300000,multiplicative,17.389750,multiplicative,52.30,NaN
48,Burkina Faso,wheat flour,vitamin a,48.00,na,52.300000,multiplicative,12.388563,regression,52.30,NaN
49,Burkina Faso,wheat flour,zinc,48.00,na,52.300000,multiplicative,12.388563,regression,52.30,NaN
59,Kenya,wheat flour,folic acid,36.58,na,30.950000,na,34.703333,multiplicative,36.58,NaN
60,Kenya,wheat flour,iron,36.58,na,30.950000,na,33.835750,multiplicative,36.58,NaN
61,Kenya,wheat flour,vitamin a,36.58,na,30.950000,na,34.703333,multiplicative,36.58,NaN
62,Kenya,wheat flour,zinc,36.58,na,30.950000,na,34.703333,multiplicative,36.58,NaN
73,Viet Nam,oil,vitamin a,38.30,na,90.000000,na,0.000000,regression,56.30,NaN


In [17]:
df[(df.B < df.C) & (df.C_estimation_status=="regression")]

,location_name,vehicle,nutrient,A,A_estimation_status,B,B_estimation_status,C,C_estimation_status,reg_A,reg_B
101,Côte d'Ivoire,wheat flour,folic acid,54.7,na,10.2,na,10.599703,regression,54.7,NaN
103,Côte d'Ivoire,wheat flour,vitamin a,54.7,na,10.2,na,10.599703,regression,54.7,NaN


In [18]:
# merge on B and filter
est = pd.concat([pd.read_csv(path) for path in paths_C])

est = est.groupby(['location_name','vehicle']).mean().reset_index()

df = df.merge(est, on = ['location_name','vehicle'], how = 'left')

draws = [f'draw_{i}' for i in range(500)]

for draw in draws:
    df.loc[(df.A < df[draw]) | 
                 (df.B < df[draw]),draw] = np.nan

df['reg_C'] = df[draws].mean(axis=1)
df = df.drop(columns = draws)

# these are rows you still dont have estimates for
# df[(df.B.isna()) & (df.reg_B.isna())]

df.loc[(df.C.isna()) | (df.C_estimation_status=='regression'),'C'] = df.reg_C

In [19]:
df[(df.B < df.C) & (df.C_estimation_status=="regression")]

,location_name,vehicle,nutrient,A,A_estimation_status,B,B_estimation_status,C,C_estimation_status,reg_A,reg_B,reg_C
32,South Africa,oil,vitamin a,99.014877,regression,56.043265,regression,56.043265,regression,90.967018,56.043265,56.043265


In [26]:
df[(df.A < df.B) | (df.B < df.C)].shape

(17, 12)

In [25]:
ref[(ref.A < ref.B) | (ref.B < ref.C)].shape

(27, 10)

In [21]:
df[(df.A < df.B)]

,location_name,vehicle,nutrient,A,A_estimation_status,B,B_estimation_status,C,C_estimation_status,reg_A,reg_B,reg_C
46,Burkina Faso,wheat flour,folic acid,48.0,na,52.30,multiplicative,32.164500,multiplicative,52.30,NaN,NaN
47,Burkina Faso,wheat flour,iron,48.0,na,52.30,multiplicative,17.389750,multiplicative,52.30,NaN,NaN
48,Burkina Faso,wheat flour,vitamin a,48.0,na,52.30,multiplicative,NaN,regression,52.30,NaN,NaN
49,Burkina Faso,wheat flour,zinc,48.0,na,52.30,multiplicative,NaN,regression,52.30,NaN,NaN
73,Viet Nam,oil,vitamin a,38.3,na,90.00,na,NaN,regression,56.30,NaN,NaN
74,Viet Nam,oil,vitamin a,56.3,regression,90.00,na,NaN,regression,56.30,NaN,NaN
79,Viet Nam,wheat flour,folic acid,16.4,na,21.93,multiplicative,4.111875,multiplicative,21.93,NaN,NaN
80,Viet Nam,wheat flour,iron,16.4,na,21.93,multiplicative,4.111875,multiplicative,21.93,NaN,NaN
81,Viet Nam,wheat flour,vitamin a,16.4,na,21.93,multiplicative,4.111875,multiplicative,21.93,NaN,NaN
82,Viet Nam,wheat flour,zinc,16.4,na,21.93,multiplicative,4.111875,multiplicative,21.93,NaN,NaN


In [22]:
df[(df.B < df.C)]

,location_name,vehicle,nutrient,A,A_estimation_status,B,B_estimation_status,C,C_estimation_status,reg_A,reg_B,reg_C
32,South Africa,oil,vitamin a,99.014877,regression,56.043265,regression,56.043265,regression,90.967018,56.043265,56.043265
45,Burkina Faso,oil,vitamin a,92.230000,na,51.975594,multiplicative,75.000000,na,92.230000,NaN,NaN
59,Kenya,wheat flour,folic acid,36.580000,na,30.950000,na,34.703333,multiplicative,36.580000,NaN,NaN
60,Kenya,wheat flour,iron,36.580000,na,30.950000,na,33.835750,multiplicative,36.580000,NaN,NaN
61,Kenya,wheat flour,vitamin a,36.580000,na,30.950000,na,34.703333,multiplicative,36.580000,NaN,NaN
62,Kenya,wheat flour,zinc,36.580000,na,30.950000,na,34.703333,multiplicative,36.580000,NaN,NaN
102,Côte d'Ivoire,wheat flour,iron,54.700000,na,10.200000,na,13.401500,multiplicative,54.700000,NaN,10.200000


In [29]:
df[(df.A.isna() | df.B.isna() | df.C.isna())]

,location_name,vehicle,nutrient,A,A_estimation_status,B,B_estimation_status,C,C_estimation_status,reg_A,reg_B,reg_C
48,Burkina Faso,wheat flour,vitamin a,48.000000,na,52.3,multiplicative,NaN,regression,52.300000,NaN,NaN
49,Burkina Faso,wheat flour,zinc,48.000000,na,52.3,multiplicative,NaN,regression,52.300000,NaN,NaN
73,Viet Nam,oil,vitamin a,38.300000,na,90.0,na,NaN,regression,56.300000,NaN,NaN
74,Viet Nam,oil,vitamin a,56.300000,regression,90.0,na,NaN,regression,56.300000,NaN,NaN
96,Côte d'Ivoire,maize flour,folic acid,84.907643,regression,NaN,regression,11.160705,regression,84.958352,NaN,11.160705
97,Côte d'Ivoire,maize flour,iron,84.907643,regression,NaN,regression,11.160705,regression,84.958352,NaN,11.160705
98,Côte d'Ivoire,maize flour,vitamin a,84.907643,regression,NaN,regression,11.160705,regression,84.958352,NaN,11.160705
99,Côte d'Ivoire,maize flour,zinc,84.907643,regression,NaN,regression,11.160705,regression,84.958352,NaN,11.160705
115,Indonesia,wheat flour,folic acid,75.728763,regression,NaN,regression,73.100000,na,75.761649,NaN,NaN
116,Indonesia,wheat flour,iron,75.728763,regression,NaN,regression,73.100000,na,75.761649,NaN,NaN


In [23]:
break

SyntaxError: 'break' outside loop (<ipython-input-23-6aaf1f276005>, line 4)

In [ ]:
fort

In [ ]:
df = ref.merge(eats, on = ['location_name','vehicle']).merge(ip, on = ['location_name','vehicle']).merge(fort, on = ['location_name','vehicle'])

In [ ]:
df.loc[df.A.isna(),'A'] = df.reg_A
df.loc[df.B.isna(),'B'] = df.reg_B
df.loc[df.C.isna(),'C'] = df.reg_C

In [ ]:
df[(df.A < df.B) | (df.B < df.C)][['location_name','vehicle','A','B','C']]

In [ ]:
chosen_pts[(chosen_pts.location_name=="Cameroon") & (chosen_pts.vehicle=="wheat flour")]

In [ ]:
chosen_pts = chosen_pts[chosen_pts.u5_applicable]
chosen_pts = chosen_pts[(chosen_pts.vehicle=="wheat flour") & (chosen_pts.estimation_status!="regression")]

In [ ]:
A = chosen_pts[(chosen_pts.value_description=="percent of population eating vehicle")].rename(columns={'value_mean':"A",'estimation_status':'A_estimation_status'})
B = chosen_pts[(chosen_pts.value_description=="percent of population eating industrially produced vehicle")].rename(columns={'value_mean':"B",'estimation_status':'B_estimation_status'})
C = chosen_pts[(chosen_pts.value_description=="percent of population eating fortified vehicle")].rename(columns={'value_mean':"C",'estimation_status':'C_estimation_status'})

A = A.drop(columns=['value_description','nutrient'])
B = B.drop(columns=['value_description','nutrient'])
C = C.drop(columns=['value_description'])

In [ ]:
ref = A.merge(B, on = ['location_id','location_name','vehicle'], how = 'left').merge(C, on = ['location_id','location_name','vehicle'], how = 'outer')[['location_name','vehicle','nutrient','A','A_estimation_status','B','B_estimation_status','C','C_estimation_status']]

ref

In [ ]:
# wheat_df = df.copy()
# wheat_df = wheat_df.loc[(wheat_df.vehicle=="wheat flour"),['location_name','percent_of_population_eating_vehicle',
#                                                'percent_of_population_eating_industrially_produced_vehicle',
#        'nutrient',
#        'percent_of_population_eating_fortified_vehicle']]

In [ ]:
wheat_estimates = wheat_estimates.groupby(['location_name']).mean().reset_index()
wheat_estimates

In [ ]:
wheat_df = ref.merge(wheat_estimates, on = 'location_name', how = 'left')
wheat_df

In [ ]:
draws = [f'draw_{i}' for i in range(500)]

for draw in draws:
    wheat_df.loc[(wheat_df.A < wheat_df[draw]) | 
                 (wheat_df.C > wheat_df[draw]),draw] = np.nan

wheat_df['est_pct_fortifiable'] = wheat_df[draws].mean(axis=1)
wheat_df = wheat_df.drop(columns = draws)

In [ ]:
pd.set_option('display.max_rows', 20)
wheat_df[(wheat_df.A < wheat_df.est_pct_fortifiable) | (wheat_df.C > wheat_df.est_pct_fortifiable)]

In [ ]:
wheat_df[wheat_df.est_pct_fortifiable.isna()]

In [ ]:
wheat_df.B_estimation_status.unique()

In [ ]:
wheat_df['combo'] = wheat_df['B']
wheat_df.loc[(wheat_df.B.isna()),'combo'] = wheat_df['est_pct_fortifiable']

In [ ]:
wheat_df[(wheat_df.A < wheat_df.combo) | (wheat_df.C > wheat_df.combo)]

In [ ]:
wheat_df[(wheat_df.est_pct_fortifiable.isna()) & (wheat_df.percent_of_population_eating_industrially_produced_vehicle.isna())]

In [ ]:
wheat_df.loc[(wheat_df.percent_of_population_eating_industrially_produced_vehicle.notna()),'est_pct_fortifiable'] = wheat_df.percent_of_population_eating_industrially_produced_vehicle

In [ ]:
wheat_df[wheat_df.est_pct_fortifiable.isna()]

In [ ]:
wheat_df.loc[(wheat_df.percent_of_population_eating_industrially_produced_vehicle > wheat_df.percent_of_population_eating_vehicle)]

In [ ]:
wheat_df.loc[(wheat_df.est_pct_fortifiable > wheat_df.percent_of_population_eating_vehicle)]

In [ ]:
wheat_df